In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import seaborn as sns

In [ ]:
os.getcwd()

In [ ]:
target_variable = "trr_5"

In [ ]:
quantiles = 10

In [ ]:
os.chdir("../../work/ktarlind/results")

In [ ]:
def remove_small_caps(results_df):
    current_results = results_df.copy()
    
    print(current_results["conviction_class"].value_counts())
    
    results_na_tested = current_results[current_results["currency"].isin(["USD", "CAD"])]
    results_row_tested = current_results[~current_results["currency"].isin(["USD", "CAD"])]
    
    print("Before cap")
    
    print(results_na_tested["market_cap_usd"].min())
    print(results_row_tested["market_cap_usd"].min())
    
    print(results_na_tested["market_cap_usd"].mean())
    print(results_row_tested["market_cap_usd"].mean())

    print(results_na_tested["gvkey"].nunique())
    print(results_row_tested["gvkey"].nunique())

    print(results_na_tested["gvkey"].value_counts()[results_na_tested["gvkey"].value_counts() >= 100/7].shape[0])
    print(results_row_tested["gvkey"].value_counts()[results_row_tested["gvkey"].value_counts() >= 100/7].shape[0])
    
    if current_results["market_cap_usd"].min() < 100000:
    
        results_na_tested = results_na_tested.groupby("date").apply(lambda x: x[x["market_cap_usd"] > x["market_cap_usd"].quantile(min_market_cap_percentile_na)]).reset_index(drop=True)
        results_row_tested = results_row_tested.groupby("date").apply(lambda x: x[x["market_cap_usd"] > x["market_cap_usd"].quantile(min_market_cap_percentile_global)]).reset_index(drop=True)

        results_na_tested = results_na_tested[results_na_tested["volume_usd_5"] > volume_usd_5_min]
        results_row_tested = results_row_tested[results_row_tested["volume_usd_5"] > volume_usd_5_min]

        current_results = pd.concat([results_na_tested, results_row_tested])
        current_results.sort_values(["date", "gvkey"], inplace=True)
        
        print("After cap")

        print(results_na_tested["market_cap_usd"].min())
        print(results_row_tested["market_cap_usd"].min())
        
        print(results_na_tested["market_cap_usd"].mean())
        print(results_row_tested["market_cap_usd"].mean())

        print(results_na_tested["gvkey"].nunique())
        print(results_row_tested["gvkey"].nunique())

        print(results_na_tested["gvkey"].value_counts()[results_na_tested["gvkey"].value_counts() >= 100/7].shape[0])
        print(results_row_tested["gvkey"].value_counts()[results_row_tested["gvkey"].value_counts() >= 100/7].shape[0])
        
        current_results["trr_5_fwd_class"] = 0
        current_results.loc[current_results.groupby(['date'])["trr_5_fwd"].transform(lambda x: x <= x.quantile(0.3333)), "trr_5_fwd_class"] = -1
        current_results.loc[current_results.groupby(['date'])["trr_5_fwd"].transform(lambda x: x >= x.quantile(0.6666)), "trr_5_fwd_class"] = 1

        
        current_results["conviction_class"] = 0
        current_results.loc[current_results.groupby(['date'])["conviction"].transform(lambda x: x <= x.quantile(0.3333)), "conviction_class"] = -1
        current_results.loc[current_results.groupby(['date'])["conviction"].transform(lambda x: x >= x.quantile(0.6666)), "conviction_class"] = 1
        print(current_results["conviction_class"].value_counts())

    else:
        print("Results already capped, not capping again")
    return current_results.copy()

In [ ]:
def add_quantiles(results_df):
    results_df = results_df.copy()
    results_df['conviction_quantile'] = pd.qcut(results_df['conviction'], quantiles, labels=False)
    results_df['top_quantile'] = pd.qcut(results_df['pred_2'], quantiles, labels=False)
    results_df['bottom_quantile'] = pd.qcut(results_df['pred_0'], quantiles, labels=False)
    return results_df.copy()

In [ ]:
def set_time_period(results_df, first_date, last_date):
    current_results = results_df.copy()
    current_results = current_results[current_results["date"] > pd.Timestamp(first_date)]
    current_results = current_results[current_results["date"] < pd.Timestamp(last_date)]
    return current_results.copy()

In [ ]:
def choose_bear_data(results_df, offset_days = 0):
    current_results = results_df.copy()
    
    current_results["date"] = pd.to_datetime(current_results["date"])
    
    bear_weeks = pd.read_csv('../bear_weeks.csv', delimiter=';', parse_dates=[1,2], infer_datetime_format=True, dayfirst=True)

    date_intervals = []
    bear_dates = []
    for index, row in bear_weeks.iterrows():
        start_date = row['monday_start']
        end_date = row['monday_end']
        date_intervals.append((pd.Timestamp(start_date) + pd.Timedelta(days=offset_days), pd.Timestamp(end_date) + pd.Timedelta(days=offset_days)))
    
    for start_date, end_date in date_intervals:
        bear_dates.extend(pd.date_range(start_date, end_date, freq='d'))

    filtered_results = current_results[current_results["date"].isin(bear_dates)]
    return filtered_results

In [ ]:
def choose_bull_data(results_df, offset_days = 0):
    current_results = results_df.copy()
    
    current_results["date"] = pd.to_datetime(current_results["date"])
    
    bull_weeks = pd.read_csv('../bull_weeks.csv', delimiter=';', parse_dates=[1,2], infer_datetime_format=True, dayfirst=True)

    date_intervals = []
    bull_dates = []
    for index, row in bull_weeks.iterrows():
        start_date = row['monday_start']
        end_date = row['monday_end']
        date_intervals.append((pd.Timestamp(start_date) + pd.Timedelta(days=offset_days), pd.Timestamp(end_date) + pd.Timedelta(days=offset_days)))
    
    for start_date, end_date in date_intervals:
        bull_dates.extend(pd.date_range(start_date, end_date, freq='d'))

    filtered_results = current_results[current_results["date"].isin(bull_dates)]
    return filtered_results

In [ ]:
def choose_non_bull_bear_data(results_df, offset_days = 0):
    current_results = results_df.copy()
    
    current_results["date"] = pd.to_datetime(current_results["date"])
    
    bear_weeks = pd.read_csv('../bear_weeks.csv', delimiter=';', parse_dates=[1,2], infer_datetime_format=True, dayfirst=True)
    bull_weeks = pd.read_csv('../bull_weeks.csv', delimiter=';', parse_dates=[1,2], infer_datetime_format=True, dayfirst=True)

    bull_bear_dates = []

    bull_date_intervals = []
    for index, row in bull_weeks.iterrows():
        start_date = row['monday_start']
        end_date = row['monday_end']
        bull_date_intervals.append((pd.Timestamp(start_date) + pd.Timedelta(days=offset_days), pd.Timestamp(end_date) + pd.Timedelta(days=offset_days)))

    for start_date, end_date in bull_date_intervals:
        bull_bear_dates.extend(pd.date_range(start_date, end_date, freq='d'))


    bear_date_intervals = []
    for index, row in bear_weeks.iterrows():
        start_date = row['monday_start']
        end_date = row['monday_end']
        bear_date_intervals.append((pd.Timestamp(start_date) + pd.Timedelta(days=offset_days), pd.Timestamp(end_date)  + pd.Timedelta(days=offset_days)))
    
    for start_date, end_date in bear_date_intervals:
        bull_bear_dates.extend(pd.date_range(start_date, end_date, freq='d'))

    filtered_results = current_results[~current_results["date"].isin(bull_bear_dates)]
    
    return filtered_results

In [ ]:
def plot_cm(df, bear_test=False, bull_test=False, rough=True, file_name = None):
    current_df = df.copy()
    
    assert(not (bull_test and bear_test))
    
    if rough:
        if bear_test:
            current_df = current_df[current_df["date"] <= pd.Timestamp("2022-10-14")]
        if bull_test:
            current_df = current_df[current_df["date"] > pd.Timestamp("2022-10-14")]

    else:
        if bear_test:
            current_df = choose_bear_data(current_df)
        if bull_test:
            current_df = choose_bull_data(current_df)

    print(current_df["date"].max())
    print(current_df["date"].min())

    cm = metrics.confusion_matrix(current_df["trr_5_fwd_class"], current_df["conviction_class"], normalize="true")

    ax = plt.subplot()
    sns.set(font_scale=1.2)
    plt_hm = sns.heatmap(cm, annot=True, ax=ax, cmap="Blues", fmt=".4f", cbar=False, linecolor="black", linewidths=1, square=True)  
    ax.set_ylabel("True values")
    ax.set_xlabel("Predicted values")
    ax.xaxis.set_ticklabels(['Under', 'Middle', "Over"]);
    ax.yaxis.set_ticklabels(['Under', 'Middle', "Over"]);
            
    if file_name is not None:
        plt_hm.figure.savefig(f'../figures/{file_name}.pdf', dpi=1000, bbox_inches='tight')
            
    return plt_hm

In [ ]:
def plot_bull_bear_mean_deciles(bull_df, bear_df, all_df):
    #BULL/BEAR TO BULL/BEAR RETURNS MEAN
    plt.rcdefaults()
    
    current_bull = bull_df.copy()
    current_bear = bear_df.copy()
    current_all  = all_df.copy()

    current_bull["trr_5_fwd"] = (1 + current_bull["trr_5_fwd"]).pow(5) - 1
    current_bear["trr_5_fwd"] = (1 + current_bear["trr_5_fwd"]).pow(5) - 1
    current_all["trr_5_fwd"] = (1 + current_all["trr_5_fwd"]).pow(5) - 1

    current_bull["trr_5"] = (1 + current_bull["trr_5"]).pow(5) - 1
    current_bear["trr_5"] = (1 + current_bear["trr_5"]).pow(5) - 1
    current_all["trr_5"] = (1 + current_all["trr_5"]).pow(5) - 1

    assert(not (na_test and global_test))

    assert(not (bull_bear_period_rough and bull_bear_period))


    if na_test:
        current_bull = current_bull[current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[current_all["currency"].isin(["USD", "CAD"])]

    if global_test:
        current_bull = current_bull[~current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[~current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[~current_all["currency"].isin(["USD", "CAD"])]


    print(current_all["date"].max())
    print(current_all["date"].min())

    if bull_bear_period_rough:
        current_bull_bull = current_bull[current_bull["date"] > pd.Timestamp("2022-10-14")]
        current_bear_bull = current_bear[current_bear["date"] > pd.Timestamp("2022-10-14")]
        current_all_bull  = current_all[current_all["date"] > pd.Timestamp("2022-10-14")]

        current_bull_bear = current_bull[current_bull["date"] <= pd.Timestamp("2022-10-14")]
        current_bear_bear = current_bear[current_bear["date"] <= pd.Timestamp("2022-10-14")]
        current_all_bear  = current_all[current_all["date"] <= pd.Timestamp("2022-10-14")]

        print(current_all_bull["date"].max())
        print(current_all_bull["date"].min())

        print(current_all_bear["date"].max())
        print(current_all_bear["date"].min())


    if bull_bear_period:
        current_bull_bull = choose_bull_data(current_bull)
        current_bear_bull = choose_bull_data(current_bear)
        current_all_bull  = choose_bull_data(current_all)

        current_bull_bear = choose_bear_data(current_bull)
        current_bear_bear = choose_bear_data(current_bear)
        current_all_bear  = choose_bear_data(current_all)

        print(current_all_bull["date"].max())
        print(current_all_bull["date"].min())

        print(current_all_bear["date"].max())
        print(current_all_bear["date"].min())


    current_bull_bull = add_quantiles(current_bull_bull)
    current_bear_bull = add_quantiles(current_bear_bull)
    current_all_bull = add_quantiles(current_all_bull)

    current_bull_bear = add_quantiles(current_bull_bear)
    current_bear_bear = add_quantiles(current_bear_bear)
    current_all_bear = add_quantiles(current_all_bear)

    current_bull = add_quantiles(current_bull)
    current_bear = add_quantiles(current_bear)
    current_all = add_quantiles(current_all)

    fig, axs = plt.subplots(3, 3, figsize=(15, 15))

    current_all.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,0])
    current_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,1])
    current_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,2])

    max_y = max(max(current_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_all.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

    min_y = min(min(current_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_all.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
    for ax in axs[0,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)


    axs[0, 0].axhline(current_all[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[0, 1].axhline(current_all[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[0, 2].axhline(current_all[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

    axs[0, 0].set_ylabel('All test', fontsize=16)

    current_all_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,0])
    current_bull_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,1])
    current_bear_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,2])

    max_y = max(max(current_bull_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_bear_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_all_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

    min_y = min(min(current_bull_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_bear_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_all_bull.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
    for ax in axs[1,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)

    axs[1, 0].axhline(current_all_bull[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[1, 1].axhline(current_all_bull[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[1, 2].axhline(current_all_bull[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

    axs[1, 0].set_ylabel('Bull test', fontsize=16)


    current_all_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,0])
    current_bull_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,1])
    current_bear_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,2])

    max_y = max(max(current_bull_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_bear_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               max(current_all_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

    min_y = min(min(current_bull_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_bear_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
               min(current_all_bear.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
    for ax in axs[2,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)

    axs[2, 0].axhline(current_all_bear[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[2, 1].axhline(current_all_bear[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
    axs[2, 2].axhline(current_all_bear[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

    axs[2, 0].set_ylabel('Bear test', fontsize=16)

    axs[0, 0].set_title("All-trained", fontsize=16)
    axs[0, 1].set_title("Bull-trained", fontsize=16)
    axs[0, 2].set_title("Bear-trained", fontsize=16)

    plt.tight_layout()
    fig.subplots_adjust(wspace=0.25)

    for ax in axs.flat:
        ax.set_xlabel("")
        ax.tick_params(axis='both', which='major', labelsize=14)
        vals = ax.get_yticks()
        print(vals)
        ax.set_yticklabels(['+{:,.2%}'.format(abs(x)) if x > 0 else '{:,.2%}'.format(x) if x == 0 else '-{:,.2%}'.format(abs(x)) for x in vals])

    return fig, ax

In [ ]:
def plot_bull_bear_geom_deciles(bull_df, bear_df, all_df):
    from scipy.stats import gmean
    #BULL/BEAR TO BULL/BEAR RETURNS GEOM
    plt.rcdefaults()
    
    current_bull = bull_df.copy()
    current_bear = bear_df.copy()
    current_all  = all_df.copy()

    current_bull["trr_5_fwd"] = (1 + current_bull["trr_5_fwd"]).pow(5) - 1
    current_bear["trr_5_fwd"] = (1 + current_bear["trr_5_fwd"]).pow(5) - 1
    current_all["trr_5_fwd"] = (1 + current_all["trr_5_fwd"]).pow(5) - 1

    current_bull["trr_5"] = (1 + current_bull["trr_5"]).pow(5) - 1
    current_bear["trr_5"] = (1 + current_bear["trr_5"]).pow(5) - 1
    current_all["trr_5"] = (1 + current_all["trr_5"]).pow(5) - 1

    assert(not (na_test and global_test))

    assert(not (bull_bear_period_rough and bull_bear_period))


    if na_test:
        current_bull = current_bull[current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[current_all["currency"].isin(["USD", "CAD"])]

    if global_test:
        current_bull = current_bull[~current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[~current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[~current_all["currency"].isin(["USD", "CAD"])]


    print(current_all["date"].max())
    print(current_all["date"].min())

    if bull_bear_period_rough:
        current_bull_bull = current_bull[current_bull["date"] > pd.Timestamp("2022-10-14")]
        current_bear_bull = current_bear[current_bear["date"] > pd.Timestamp("2022-10-14")]
        current_all_bull  = current_all[current_all["date"] > pd.Timestamp("2022-10-14")]

        current_bull_bear = current_bull[current_bull["date"] <= pd.Timestamp("2022-10-14")]
        current_bear_bear = current_bear[current_bear["date"] <= pd.Timestamp("2022-10-14")]
        current_all_bear  = current_all[current_all["date"] <= pd.Timestamp("2022-10-14")]

        print(current_all_bull["date"].max())
        print(current_all_bull["date"].min())

        print(current_all_bear["date"].max())
        print(current_all_bear["date"].min())


    if bull_bear_period:
        current_bull_bull = choose_bull_data(current_bull)
        current_bear_bull = choose_bull_data(current_bear)
        current_all_bull  = choose_bull_data(current_all)

        current_bull_bear = choose_bear_data(current_bull)
        current_bear_bear = choose_bear_data(current_bear)
        current_all_bear  = choose_bear_data(current_all)

        print(current_all_bull["date"].max())
        print(current_all_bull["date"].min())

        print(current_all_bear["date"].max())
        print(current_all_bear["date"].min())


    current_bull_bull = add_quantiles(current_bull_bull)
    current_bear_bull = add_quantiles(current_bear_bull)
    current_all_bull = add_quantiles(current_all_bull)

    current_bull_bear = add_quantiles(current_bull_bear)
    current_bear_bear = add_quantiles(current_bear_bear)
    current_all_bear = add_quantiles(current_all_bear)

    current_bull = add_quantiles(current_bull)
    current_bear = add_quantiles(current_bear)
    current_all = add_quantiles(current_all)

    fig, axs = plt.subplots(3, 3, figsize=(15, 15))

    gmean_quantiles_all = ((current_all.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_all.plot(kind="bar", grid=True, ax=axs[0,0])

    gmean_quantiles_bull = ((current_bull.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bull.plot(kind="bar", grid=True, ax=axs[0,1])

    gmean_quantiles_bear = ((current_bear.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bear.plot(kind="bar", grid=True, ax=axs[0,2])

    max_y = max(max(gmean_quantiles_all),
               max(gmean_quantiles_bull),
               max(gmean_quantiles_bear))

    min_y = min(min(gmean_quantiles_all),
               min(gmean_quantiles_bull),
               min(gmean_quantiles_bear))
    for ax in axs[0,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)


    axs[0, 0].axhline(gmean(current_all.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[0, 1].axhline(gmean(current_all.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[0, 2].axhline(gmean(current_all.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")

    axs[0, 0].set_ylabel('All test', fontsize=16)

    gmean_quantiles_all_bull = ((current_all_bull.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_all_bull.plot(kind="bar", grid=True, ax=axs[1,0])

    gmean_quantiles_bull_bull = ((current_bull_bull.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bull_bull.plot(kind="bar", grid=True, ax=axs[1,1])

    gmean_quantiles_bear_bull = ((current_bear_bull.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bear_bull.plot(kind="bar", grid=True, ax=axs[1,2])

    max_y = max(max(gmean_quantiles_all_bull),
               max(gmean_quantiles_bull_bull),
               max(gmean_quantiles_bear_bull))

    min_y = min(min(gmean_quantiles_all_bull),
               min(gmean_quantiles_bull_bull),
               min(gmean_quantiles_bear_bull))
    for ax in axs[1,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)

    axs[1, 0].axhline(gmean(current_all_bull.groupby("date")[target_variable + "_fwd"].mean() + 1) - 1, label="avg trr_5_fwd")
    axs[1, 1].axhline(gmean(current_all_bull.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[1, 2].axhline(gmean(current_all_bull.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[1, 0].set_ylabel('Bull test', fontsize=16)


    gmean_quantiles_all_bear = ((current_all_bear.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_all_bear.plot(kind="bar", grid=True, ax=axs[2,0]) 

    gmean_quantiles_bull_bear = ((current_bull_bear.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bull_bear.plot(kind="bar", grid=True, ax=axs[2,1])

    gmean_quantiles_bear_bear = ((current_bear_bear.groupby(["date","conviction_quantile"])[target_variable + "_fwd"].mean() + 1).groupby("conviction_quantile").apply(gmean) - 1)
    gmean_quantiles_bear_bear.plot(kind="bar", grid=True, ax=axs[2,2])

    max_y = max(max(gmean_quantiles_all_bear),
               max(gmean_quantiles_bull_bear),
               max(gmean_quantiles_bear_bear))

    min_y = min(min(gmean_quantiles_all_bear),
               min(gmean_quantiles_bull_bear),
               min(gmean_quantiles_bear_bear))
    for ax in axs[2,:]:
        ax.set_ylim(min_y - 0.001, max_y + 0.001)

    axs[2, 0].axhline(gmean(current_all_bear.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[2, 1].axhline(gmean(current_all_bear.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")
    axs[2, 2].axhline(gmean(current_all_bear.groupby("date")[target_variable + "_fwd"].mean() +1) - 1, label="avg trr_5_fwd")

    axs[2, 0].set_ylabel('Bear test', fontsize=16)

    axs[0, 0].set_title("All-trained", fontsize=16)
    axs[0, 1].set_title("Bull-trained", fontsize=16)
    axs[0, 2].set_title("Bear-trained", fontsize=16)

    plt.tight_layout()
    fig.subplots_adjust(wspace=0.25)

    for ax in axs.flat:
        ax.set_xlabel("")
        ax.tick_params(axis='both', which='major', labelsize=14)
        vals = ax.get_yticks()
        ax.set_yticklabels(['+{:,.2%}'.format(abs(x)) if x > 0 else '{:,.2%}'.format(x) if x == 0 else '-{:,.2%}'.format(abs(x)) for x in vals])


    return fig, ax

In [ ]:
def plot_cumulative_returns(df_bull, df_bear, df_all, both_adaptive = False, return_min = None, return_max = None, trade_index = False):
    from matplotlib.ticker import MultipleLocator
    import matplotlib.dates as mdates

    current_bull = df_bull.copy()
    current_bear = df_bear.copy()
    current_all = df_all.copy()

    current_bull["date"] = pd.to_datetime(current_bull["date"])
    current_bear["date"] = pd.to_datetime(current_bear["date"])
    current_all["date"] = pd.to_datetime(current_all["date"])


    plt.rcdefaults()
    fig, ax = plt.subplots(figsize=(20, 10))


    days_offset = 7 #Because trr_5_fwd is 7 days in the future, shift dates forward 7 days

    assert(not (na_test and global_test))


    if na_test:
        current_bull = current_bull[current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[current_all["currency"].isin(["USD", "CAD"])]
        print("na_test")

    if global_test:
        current_bull = current_bull[~current_bull["currency"].isin(["USD", "CAD"])]
        current_bear = current_bear[~current_bear["currency"].isin(["USD", "CAD"])]
        current_all  = current_all[~current_all["currency"].isin(["USD", "CAD"])]
        print("global_test")


    print(current_all["date"].max())
    print(current_all["date"].min())

    current_bull = add_quantiles(current_bull)
    current_bear = add_quantiles(current_bear)
    current_all = add_quantiles(current_all)

    current_bull["trr_5_fwd_tot"] = (1 + current_bull["trr_5_fwd"]).pow(5) - 1
    current_bear["trr_5_fwd_tot"] = (1 + current_bear["trr_5_fwd"]).pow(5) - 1
    current_all["trr_5_fwd_tot"] = (1 + current_all["trr_5_fwd"]).pow(5) - 1

    if return_min is not None:
        current_bull = current_bull[current_bull["trr_5_fwd_tot"] >= return_min]
        current_bear = current_bear[current_bear["trr_5_fwd_tot"] >= return_min]
        current_all = current_all[current_all["trr_5_fwd_tot"] >= return_min]

    if return_max is not None:
        current_bull = current_bull[current_bull["trr_5_fwd_tot"] <= return_max]
        current_bear = current_bear[current_bear["trr_5_fwd_tot"] <= return_max]
        current_all = current_all[current_all["trr_5_fwd_tot"] <= return_max]


    current_all_returns = (current_all[current_all["conviction_quantile"].isin(buy_deciles)].groupby("date")["trr_5_fwd_tot"].mean() -\
        current_all[current_all["conviction_quantile"].isin(sell_deciles)].groupby("date")["trr_5_fwd_tot"].mean()).reset_index()

    current_bear_returns = (current_bear[current_bear["conviction_quantile"].isin(buy_deciles)].groupby("date")["trr_5_fwd_tot"].mean() -\
        current_bear[current_bear["conviction_quantile"].isin(sell_deciles)].groupby("date")["trr_5_fwd_tot"].mean()).reset_index()

    current_bull_returns = (current_bull[current_bull["conviction_quantile"].isin(buy_deciles)].groupby("date")["trr_5_fwd_tot"].mean() -\
        current_bull[current_bull["conviction_quantile"].isin(sell_deciles)].groupby("date")["trr_5_fwd_tot"].mean()).reset_index()

    current_index = current_all.copy()

    current_index_returns = current_index.groupby("date")["trr_5_fwd_tot"].mean().reset_index()

    current_all_returns["trr_5_fwd_tot"] = current_all_returns["trr_5_fwd_tot"] - costs
    current_bear_returns["trr_5_fwd_tot"] = current_bear_returns["trr_5_fwd_tot"] - costs
    current_bull_returns["trr_5_fwd_tot"] = current_bull_returns["trr_5_fwd_tot"] - costs

    current_all_returns["date"] = current_all_returns["date"] + pd.Timedelta(days=days_offset)
    current_bear_returns["date"] = current_bear_returns["date"] + pd.Timedelta(days=days_offset)
    current_bull_returns["date"] = current_bull_returns["date"] + pd.Timedelta(days=days_offset)
    current_index_returns["date"] = current_index_returns["date"] + pd.Timedelta(days=days_offset)

    current_all_returns = current_all_returns.append({"trr_5_fwd_tot": 0, "date": current_all_returns["date"].min() - pd.Timedelta(days=7)}, ignore_index=True).sort_values("date")
    current_bear_returns = current_bear_returns.append({"trr_5_fwd_tot": 0, "date": current_bear_returns["date"].min() - pd.Timedelta(days=7)}, ignore_index=True).sort_values("date")
    current_bull_returns = current_bull_returns.append({"trr_5_fwd_tot": 0, "date": current_bull_returns["date"].min() - pd.Timedelta(days=7)}, ignore_index=True).sort_values("date")
    current_index_returns = current_index_returns.append({"trr_5_fwd_tot": 0, "date": current_index_returns["date"].min() - pd.Timedelta(days=7)}, ignore_index=True).sort_values("date")

    current_bull["date"] = current_bull["date"].dt.date
    current_bear["date"] = current_bear["date"].dt.date
    current_all["date"] = current_all["date"].dt.date
    current_index_returns["date"] = current_index_returns["date"].dt.date


    ((current_index_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="Index",
                                                                                color="black",
                                                                                linestyle='--',
                                                                                ax=ax)
    if trade_index:
        current_trade_index_returns = choose_bear_data(current_index_returns)
        current_trade_index_returns["trr_5_fwd_tot"] = - current_trade_index_returns["trr_5_fwd_tot"]

        current_trade_index_returns = pd.concat([current_trade_index_returns, choose_bull_data(current_index_returns, 0), choose_non_bull_bear_data(current_index_returns, 0)]).sort_values("date")


        ((current_trade_index_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                    label="Index, adaptive",
                                                                                    color="black",
                                                                                    ax=ax)
    ((current_all_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="All-trained",
                                                                                color="tab:green",
                                                                                ax=ax)
    ((current_bear_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="Bear-trained",
                                                                                color="tab:red",
                                                                                ax=ax)
    ((current_bull_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="Bull-trained", 
                                                                                color="tab:blue",
                                                                                ax=ax)
    if both_adaptive:
        current_adaptive_rough_returns = pd.concat([current_bull_returns[current_bull_returns["date"] > (pd.Timestamp("2022-10-14") + pd.Timedelta(days=adaptive_latency_days))], 
                                               current_bear_returns[current_bear_returns["date"] <= (pd.Timestamp("2022-10-14") + pd.Timedelta(days=adaptive_latency_days))]]).reset_index().sort_values("date")

        current_adaptive_returns = pd.concat([choose_bull_data(current_bull_returns, 0), choose_bear_data(current_bear_returns, 0), choose_non_bull_bear_data(current_all_returns, 0)]).sort_values("date")
                
        current_adaptive_rough_returns["date"] = pd.to_datetime(current_adaptive_rough_returns["date"]).dt.date
        current_adaptive_returns["date"] = pd.to_datetime(current_adaptive_returns["date"]).dt.date
        
        ((current_adaptive_rough_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="Adaptive, rough",
                                                                                color="tab:purple",
                                                                                ax=ax)
        
        ((current_adaptive_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                label="Adaptive",
                                                                                color="tab:pink",
                                                                                ax=ax)
        
    else:
        if rough_adaptive_model_choice:
            current_adaptive_returns = pd.concat([current_bull_returns.reset_index()[current_bull_returns.reset_index()["date"] > (pd.Timestamp("2022-10-14") + pd.Timedelta(days=adaptive_latency_days))], 
                                                   current_bear_returns.reset_index()[current_bear_returns.reset_index()["date"] <= (pd.Timestamp("2022-10-14") + pd.Timedelta(days=adaptive_latency_days))]]).sort_values("date")
        else:
            current_adaptive_returns = pd.concat([choose_bull_data(current_bull_returns.reset_index(), days_offset), choose_bear_data(current_bear_returns.reset_index(), 0), choose_non_bull_bear_data(current_all_returns.reset_index(), 0)]).sort_values("date")
        
        current_adaptive_returns["date"] = pd.to_datetime(current_adaptive_returns["date"]).dt.date

        ((current_adaptive_returns.groupby("date")["trr_5_fwd_tot"].mean() + 1).cumprod() - 1).plot(figsize=(20, 10), 
                                                                                    label="Adaptive",
                                                                                    color="tab:purple",
                                                                                    ax=ax)
        


    ax.lines[0].set_linewidth(3)
    ax.lines[1].set_linewidth(3)
    ax.lines[2].set_linewidth(3)
    ax.lines[3].set_linewidth(3)
    ax.lines[4].set_linewidth(3)
    
    if both_adaptive:
        ax.lines[5].set_linewidth(3)
        if trade_index:
            ax.lines[6].set_linewidth(3)
    elif trade_index:
        ax.lines[5].set_linewidth(3)
        
    

    plt.axvline(x=(pd.Timestamp('2022-10-14')), color="grey", linestyle="--", linewidth=3, label="Last bear friday")


    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
    ax.xaxis.set_major_formatter((mdates.DateFormatter('%b\n%Y')))
    ax.xaxis.set_minor_formatter((mdates.DateFormatter('%b')))
    
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))

    plt.grid(True, which="both", axis="both")

    ax.set_xlim(pd.Timestamp('2022-01-07'), pd.Timestamp('2023-08-30'))

    plt.legend(fontsize=16)
    
    ax.set_xlabel("")
    vals = ax.get_yticks()
    ax.set_yticklabels(['+{:,.2%}'.format(abs(x)) if x > 0 else '{:,.2%}'.format(x) if x == 0 else '-{:,.2%}'.format(abs(x)) for x in vals])

    ax.tick_params(axis='both', which='major', labelsize=16)
    ax.tick_params(axis='both', which='minor', labelsize=16)
    
    return fig, ax

In [ ]:
min_market_cap_percentile_na = 0.6
min_market_cap_percentile_global = 0.65
volume_usd_5_min = 1000000

## CATBOOST

In [ ]:
results_na = pd.read_parquet("catboost_na_2023-11-22-05-46-21_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_na = remove_small_caps(results_na)
results_na = set_time_period(results_na, "2022-01-01", "2023-08-25")

In [ ]:
results_g = pd.read_parquet("catboost_g_2023-11-23-14-30-07_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_g = remove_small_caps(results_g)
results_g = set_time_period(results_g, "2022-01-01", "2023-08-25")

In [ ]:
results_gna = pd.read_parquet("catboost_gna_2023-11-23-11-55-52_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_gna = remove_small_caps(results_gna)
results_gna = set_time_period(results_gna, "2022-01-01", "2023-08-25")

In [ ]:
results_na_detached = pd.read_parquet("catboost_NA_2023-12-08-18-39-20_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_na_detached = remove_small_caps(results_na_detached)
results_na_detached = set_time_period(results_na_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_gna_detached = pd.read_parquet("catboost_Global_2023-12-08-20-20-26_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_gna_detached = remove_small_caps(results_gna_detached)
results_gna_detached = set_time_period(results_gna_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_g_detached = pd.read_parquet("catboost_ROW_2023-12-08-20-11-01_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_g_detached = remove_small_caps(results_g_detached)
results_g_detached = set_time_period(results_g_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_na_bear = pd.read_parquet("catboost_NA_2023-12-08-19-01-28_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_na_bear = remove_small_caps(results_na_bear)
results_na_bear = set_time_period(results_na_bear, "2022-01-01", "2023-08-25")

In [ ]:
results_gna_bear = pd.read_parquet("catboost_Global_2023-12-08-19-24-25_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_gna_bear = remove_small_caps(results_gna_bear)
results_gna_bear = set_time_period(results_gna_bear, "2022-01-01", "2023-08-25")

In [ ]:
results_g_bear = pd.read_parquet("catboost_ROW_2023-12-08-19-28-31_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_g_bear = remove_small_caps(results_g_bear)
results_g_bear = set_time_period(results_g_bear, "2022-01-01", "2023-08-25")

In [ ]:
results_na_bull = pd.read_parquet("catboost_NA_2023-12-08-19-03-49_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_na_bull = remove_small_caps(results_na_bull)
results_na_bull = set_time_period(results_na_bull, "2022-01-01", "2023-08-25")

In [ ]:
results_gna_bull = pd.read_parquet("catboost_Global_2023-12-08-19-17-12_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_gna_bull = remove_small_caps(results_gna_bull)
results_gna_bull = set_time_period(results_gna_bull, "2022-01-01", "2023-08-25")

In [ ]:
results_g_bull = pd.read_parquet("catboost_ROW_2023-12-08-19-08-29_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_g_bull = remove_small_caps(results_g_bull)
results_g_bull = set_time_period(results_g_bull, "2022-01-01", "2023-08-25")

In [ ]:
#REGION TO REGION
plt.rcdefaults()
current_global = results_gna_detached.copy()
current_na = results_na_detached.copy()
current_row  = results_g_detached.copy()
title = "Regions Catboost"

bull_period_rough = False
bear_period_rough = False
bull_period = False
bear_period = False

assert(not (bull_period and bear_period))
assert(not (bull_period_rough and bear_period_rough))

current_global["trr_5_fwd"] = (1 + current_global["trr_5_fwd"]).pow(5) - 1
current_na["trr_5_fwd"] = (1 + current_na["trr_5_fwd"]).pow(5) - 1
current_row["trr_5_fwd"] = (1 + current_row["trr_5_fwd"]).pow(5) - 1


print(current_global["date"].max())
print(current_global["date"].min())

if bull_period_rough:
    current_bull = current_bull[current_bull["date"] > pd.Timestamp("2022-10-14")]
    current_bear = current_bear[current_bear["date"] > pd.Timestamp("2022-10-14")]
    current_all  = current_all[current_all["date"] > pd.Timestamp("2022-10-14")]
    
    print(current_all["date"].max())
    print(current_all["date"].min())

if bear_period_rough:
    current_bull = current_bull[current_bull["date"] < pd.Timestamp("2022-10-14")]
    current_bear = current_bear[current_bear["date"] < pd.Timestamp("2022-10-14")]
    current_all  = current_all[current_all["date"] < pd.Timestamp("2022-10-14")]
    
    print(current_all["date"].max())
    print(current_all["date"].min())
    
if bear_period:
    current_bull = choose_bear_data(current_bull)
    current_bear = choose_bear_data(current_bear)
    current_all = choose_bear_data(current_all)
    
    print(current_all["date"].max())
    print(current_all["date"].min())
    
if bull_period:
    current_bull = choose_bull_data(current_bull)
    current_bear = choose_bull_data(current_bear)
    current_all = choose_bull_data(current_all)
    
    print(current_all["date"].max())
    print(current_all["date"].min())

current_global_na = current_global[current_global["currency"].isin(["USD", "CAD"])]
current_na_na = current_na[current_na["currency"].isin(["USD", "CAD"])]
current_row_na  = current_row[current_row["currency"].isin(["USD", "CAD"])]

current_global_row = current_global[~current_global["currency"].isin(["USD", "CAD"])]
current_na_row = current_na[~current_na["currency"].isin(["USD", "CAD"])]
current_row_row  = current_row[~current_row["currency"].isin(["USD", "CAD"])]

current_global_na = add_quantiles(current_global_na)
current_na_na = add_quantiles(current_na_na)
current_row_na = add_quantiles(current_row_na)

current_global_row = add_quantiles(current_global_row)
current_na_row = add_quantiles(current_na_row)
current_row_row = add_quantiles(current_row_row)

current_global = add_quantiles(current_global)
current_na = add_quantiles(current_na)
current_row = add_quantiles(current_row)

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,0])

current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,1])

current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,2])

max_y = max(max(current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[0,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)


axs[0, 0].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[0, 1].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[0, 2].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[0, 0].set_ylabel('Global test', fontsize=16)

current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,0])
current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,1])
current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,2])

max_y = max(max(current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[1,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)

axs[1, 0].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[1, 1].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[1, 2].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[1, 0].set_ylabel('NA test', fontsize=16)


current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,0])
current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,1])
current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,2])

max_y = max(max(current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[2,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)

axs[2, 0].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[2, 1].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[2, 2].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[2, 0].set_ylabel('ROW test', fontsize=16)

axs[0, 0].set_title("Global train", fontsize=16)
axs[0, 1].set_title("NA train", fontsize=16)
axs[0, 2].set_title("ROW train", fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0.25)

for ax in axs.flat:
    ax.set_xlabel("")
    ax.tick_params(axis='both', which='major', labelsize=14)
    vals = ax.get_yticks()
    print(vals)
    ax.set_yticklabels(['+{:,.2%}'.format(abs(x)) if x > 0 else '{:,.2%}'.format(x) if x == 0 else '-{:,.2%}'.format(abs(x)) for x in vals])

In [ ]:
na_test = False
global_test = False

bull_bear_period_rough = True
bull_bear_period = False

In [ ]:
#BULL/BEAR TO BULL/BEAR RETURNS
plt.rcdefaults()

current_bull = results_gna_bull.copy()
current_bear = results_gna_bear.copy()
current_all  = results_gna_detached.copy()

assert(not (na_test and global_test))

assert(not (bull_bear_period_rough and bull_bear_period))

In [ ]:
fig, ax = plot_bull_bear_mean_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesCatboostNABearBullRough.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
fig, ax = plot_bull_bear_geom_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesGeomCatboostNABearBullRough.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
buy_deciles = [9]
sell_deciles = [0]

rough_adaptive_model_choice = False

adaptive_latency_days = 0

costs = 1 * (1/100)

fig, ax = plot_cumulative_returns(current_bull, current_bear, current_all, both_adaptive=True, trade_index=True)

#fig.savefig("../figures/CumulativeReturnsCatboostGNATop3Cost1BothAdaptive.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
plt_cm = plot_cm(current_all, bear_test=True, rough=True)
plt_cm.figure.savefig(f'../figures/CMCatboostGNAAllBearRough.pdf', dpi=1000, bbox_inches='tight')

## XGBOOST

In [ ]:
results_xgb_gna_detached = pd.read_parquet("xgboost_Global_2023-12-08-13-27-00_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_gna_detached = remove_small_caps(results_xgb_gna_detached)
results_xgb_gna_detached = set_time_period(results_xgb_gna_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_xgb_gna_bull = pd.read_parquet("xgboost_Global_2023-12-08-15-27-52_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_gna_bull = remove_small_caps(results_xgb_gna_bull)
results_xgb_gna_bull = set_time_period(results_xgb_gna_bull, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_gna_bear = pd.read_parquet("xgboost_Global_2023-12-08-14-08-35_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_gna_bear = remove_small_caps(results_xgb_gna_bear)
results_xgb_gna_bear = set_time_period(results_xgb_gna_bear, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_na_detached = pd.read_parquet("xgboost_NA_2023-12-08-18-09-30_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_na_detached = remove_small_caps(results_xgb_na_detached)
results_xgb_na_detached = set_time_period(results_xgb_na_detached, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_na_bull = pd.read_parquet("xgboost_NA_2023-12-08-18-34-21_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_na_bull = remove_small_caps(results_xgb_na_bull)
results_xgb_na_bull = set_time_period(results_xgb_na_bull, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_na_bear = pd.read_parquet("xgboost_NA_2023-12-08-18-29-18_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_na_bear = remove_small_caps(results_xgb_na_bear)
results_xgb_na_bear = set_time_period(results_xgb_na_bear, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_g_detached = pd.read_parquet("xgboost_ROW_2023-12-08-17-19-04_detached_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_g_detached = remove_small_caps(results_xgb_g_detached)
results_xgb_g_detached = set_time_period(results_xgb_g_detached, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_g_bull = pd.read_parquet("xgboost_ROW_2023-12-08-17-16-38_detached_bull_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_g_bull = remove_small_caps(results_xgb_g_bull)
results_xgb_g_bull = set_time_period(results_xgb_g_bull, "2022-01-01", "2023-08-25")


In [ ]:
results_xgb_g_bear = pd.read_parquet("xgboost_ROW_2023-12-08-17-06-52_detached_bear_trained_no_validation_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_xgb_g_bear = remove_small_caps(results_xgb_g_bear)
results_xgb_g_bear = set_time_period(results_xgb_g_bear, "2022-01-01", "2023-08-25")


In [ ]:
na_test = False
global_test = False
bull_bear_period = False
bull_bear_period_rough = True

In [ ]:
current_bull = results_xgb_gna_bull.copy()
current_bear = results_xgb_gna_bear.copy()
current_all = results_xgb_gna_detached.copy()

In [ ]:
fig, ax = plot_bull_bear_mean_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesXGBoostGNABearBullRough.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
fig, ax = plot_bull_bear_geom_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesGeomXGBoostGNABearBullRough.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
buy_deciles = [9]
sell_deciles = [0]

rough_adaptive_model_choice = False

adaptive_latency_days = 0

costs = 1 * (1/100)

fig, ax = plot_cumulative_returns(current_bull, current_bear, current_all, both_adaptive=True, trade_index=True)

#fig.savefig("../figures/CumulativeReturnsXGBoostGNATop1Cost1BothAdaptive.pdf", dpi=1000, bbox_inches='tight')

## LOGREG

In [ ]:
results_log_gna_detached = pd.read_parquet("logreg_Global_2023-12-08-04-50-36_detached_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_gna_detached = remove_small_caps(results_log_gna_detached)
results_log_gna_detached = set_time_period(results_log_gna_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_log_gna_bull = pd.read_parquet("logreg_Global_2023-12-08-10-49-50_detached_bull_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_gna_bull = remove_small_caps(results_log_gna_bull)
results_log_gna_bull = set_time_period(results_log_gna_bull, "2022-01-01", "2023-08-25")

In [ ]:
results_log_gna_bear = pd.read_parquet("logreg_Global_2023-12-08-10-28-17_detached_bear_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_gna_bear = remove_small_caps(results_log_gna_bear)
results_log_gna_bear = set_time_period(results_log_gna_bear, "2022-01-01", "2023-08-25")

In [ ]:
results_log_na_detached = pd.read_parquet("logreg_NA_2023-12-08-02-53-16_detached_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_na_detached = remove_small_caps(results_log_na_detached)
results_log_na_detached = set_time_period(results_log_na_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_log_na_bull = pd.read_parquet("logreg_NA_2023-12-08-00-58-30_detached_bull_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_na_bull = remove_small_caps(results_log_na_bull)
results_log_na_bull = set_time_period(results_log_na_bull, "2022-01-01", "2023-08-25")

In [ ]:
results_log_na_bear = pd.read_parquet("logreg_NA_2023-12-08-02-19-04_detached_bear_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_na_bear = remove_small_caps(results_log_na_bear)
results_log_na_bear = set_time_period(results_log_na_bear, "2022-01-01", "2023-08-25")

In [ ]:
results_log_g_detached = pd.read_parquet("logreg_ROW_2023-12-08-06-39-14_detached_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_g_detached = remove_small_caps(results_log_g_detached)
results_log_g_detached = set_time_period(results_log_g_detached, "2022-01-01", "2023-08-25")

In [ ]:
results_log_g_bull = pd.read_parquet("logreg_ROW_2023-12-08-10-54-22_detached_bull_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_g_bull = remove_small_caps(results_log_g_bull)
results_log_g_bull = set_time_period(results_log_g_bull, "2022-01-01", "2023-08-25")

In [ ]:
results_log_g_bear = pd.read_parquet("logreg_ROW_2023-12-08-12-13-09_detached_bear_trained_min_vol_5_1000000_min_mcap_percentile_na_0.6_min_mcap_percentile_global_0.65/results.parquet", engine="pyarrow")
results_log_g_bear = remove_small_caps(results_log_g_bear)
results_log_g_bear = set_time_period(results_log_g_bear, "2022-01-01", "2023-08-25")

In [ ]:
na_test = True
global_test = False

bull_bear_period_rough = True
bull_bear_period = False

In [ ]:
current_bull = results_log_gna_bull.copy()
current_bear = results_log_gna_bear.copy()
current_all = results_log_gna_detached.copy()

In [ ]:
fig, ax = plot_bull_bear_mean_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesLogregNABearBull.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
fig, ax = plot_bull_bear_geom_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesGeomLogregNABearBullRough.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
buy_deciles = [9]
sell_deciles = [0]

rough_adaptive_model_choice = False

adaptive_latency_days = 0

costs = 1 * (1/100)

fig, ax = plot_cumulative_returns(current_bull, current_bear, current_all, both_adaptive=True, trade_index=True)

#fig.savefig("../figures/CumulativeReturnsLogregNATop1Cost1BothAdaptive.pdf", dpi=1000, bbox_inches='tight')

## Ensemble Boosting

In [ ]:
def create_ensemble(df_1, df_2):
    current_df_1 = df_1.copy()
    current_df_2 = df_2.copy()
    
    current_ensemble = df_1.merge(df_2, on=["date", "gvkey"], suffixes=["", "_right"])

    current_ensemble["pred_2"] = (current_ensemble["pred_2"] + current_ensemble["pred_2_right"]) / 2
    current_ensemble["pred_1"] = (current_ensemble["pred_1"] + current_ensemble["pred_1_right"]) / 2
    current_ensemble["pred_0"] = (current_ensemble["pred_0"] + current_ensemble["pred_0_right"]) / 2
    current_ensemble["conviction"] = (current_ensemble["conviction"] + current_ensemble["conviction_right"]) / 2

    keep_cols = current_ensemble.columns[current_ensemble.columns.str[-6:] != "_right"]
    current_ensemble = current_ensemble[keep_cols.tolist()]

    #current_ensemble = add_quantiles(current_ensemble)
    
    return current_ensemble

In [ ]:
results_gna_detached_ensemble_boosting = create_ensemble(results_gna_detached, results_xgb_gna_detached)

In [ ]:
results_gna_bull_ensemble_boosting = create_ensemble(results_gna_bull, results_xgb_gna_bull)

In [ ]:
results_gna_bear_ensemble_boosting = create_ensemble(results_gna_bear, results_xgb_gna_bear)

In [ ]:
results_na_detached_ensemble_boosting = create_ensemble(results_na_detached, results_xgb_na_detached)

In [ ]:
results_na_bull_ensemble_boosting = create_ensemble(results_na_bull, results_xgb_na_bull)

In [ ]:
results_na_bear_ensemble_boosting = create_ensemble(results_na_bear, results_xgb_na_bear)

In [ ]:
results_g_detached_ensemble_boosting = create_ensemble(results_g_detached, results_xgb_g_detached)

In [ ]:
results_g_bull_ensemble_boosting = create_ensemble(results_g_bull, results_xgb_g_bull)

In [ ]:
results_g_bear_ensemble_boosting = create_ensemble(results_g_bear, results_xgb_g_bear)

## Ensemble boosting + log

In [ ]:
results_gna_detached_ensemble = create_ensemble(results_gna_detached_ensemble_boosting, results_log_gna_detached)

In [ ]:
results_gna_bull_ensemble = create_ensemble(results_gna_bull_ensemble_boosting, results_log_gna_bull)


In [ ]:
results_gna_bear_ensemble = create_ensemble(results_gna_bear_ensemble_boosting, results_log_gna_bear)


In [ ]:
results_na_detached_ensemble = create_ensemble(results_na_detached_ensemble_boosting, results_log_na_detached)


In [ ]:
results_na_bull_ensemble = create_ensemble(results_na_bull_ensemble_boosting, results_log_na_bull)


In [ ]:
results_na_bear_ensemble = create_ensemble(results_na_bear_ensemble_boosting, results_log_na_bear)


In [ ]:
results_g_detached_ensemble = create_ensemble(results_g_detached_ensemble_boosting, results_log_g_detached)


In [ ]:
results_g_bull_ensemble = create_ensemble(results_g_bull_ensemble_boosting, results_log_g_bull)


In [ ]:
results_g_bear_ensemble = create_ensemble(results_g_bear_ensemble_boosting, results_g_bear_ensemble_boosting)


In [ ]:
quantiles = 10

In [ ]:
na_test = True
global_test = False
bull_bear_period = True
bull_bear_period_rough = False

In [ ]:
#BULL/BEAR TO BULL/BEAR
plt.rcdefaults()
current_bull = results_na_bull_ensemble.copy()
current_bear = results_na_bear_ensemble.copy()
current_all  = results_na_detached_ensemble.copy()

assert(not (na_test and global_test))

assert(not (bull_bear_period_rough and bull_bear_period))


In [ ]:
fig, ax = plot_bull_bear_mean_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesEnsembleNABearBull.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
fig, ax = plot_bull_bear_geom_deciles(current_bull, current_bear, current_all)

#fig.savefig("../figures/QuantilesGeomEnsembleNABearBull.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
buy_deciles = [9]
sell_deciles = [0]

rough_adaptive_model_choice = False

adaptive_latency_days = 0

costs = 1 * (1/100)

fig, ax = plot_cumulative_returns(current_bull, current_bear, current_all, both_adaptive=True, trade_index=True)

#fig.savefig("../figures/CumulativeReturnsEnsembleNATop1Cost1BothAdaptive.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
#REGION TO REGION
plt.rcdefaults()
current_global = results_gna_detached.copy()
current_na = results_na_detached.copy()
current_row  = results_g_detached.copy()
title = "Regions Catboost"

bull_period_rough = False
bear_period_rough = False
bull_period = False
bear_period = False

assert(not (bull_period and bear_period))
assert(not (bull_period_rough and bear_period_rough))

current_global["trr_5_fwd"] = (1 + current_global["trr_5_fwd"]).pow(5) - 1
current_na["trr_5_fwd"] = (1 + current_na["trr_5_fwd"]).pow(5) - 1
current_row["trr_5_fwd"] = (1 + current_row["trr_5_fwd"]).pow(5) - 1


print(current_global["date"].max())
print(current_global["date"].min())

if bull_period_rough:
    current_bull = current_bull[current_bull["date"] > pd.Timestamp("2022-10-14")]
    current_bear = current_bear[current_bear["date"] > pd.Timestamp("2022-10-14")]
    current_all  = current_all[current_all["date"] > pd.Timestamp("2022-10-14")]
    
    print(current_all["date"].max())
    print(current_all["date"].min())

if bear_period_rough:
    current_bull = current_bull[current_bull["date"] < pd.Timestamp("2022-10-14")]
    current_bear = current_bear[current_bear["date"] < pd.Timestamp("2022-10-14")]
    current_all  = current_all[current_all["date"] < pd.Timestamp("2022-10-14")]
    
    print(current_all["date"].max())
    print(current_all["date"].min())
    
if bear_period:
    current_bull = choose_bear_data(current_bull)
    current_bear = choose_bear_data(current_bear)
    current_all = choose_bear_data(current_all)
    
    print(current_all["date"].max())
    print(current_all["date"].min())
    
if bull_period:
    current_bull = choose_bull_data(current_bull)
    current_bear = choose_bull_data(current_bear)
    current_all = choose_bull_data(current_all)
    
    print(current_all["date"].max())
    print(current_all["date"].min())

current_global_na = current_global[current_global["currency"].isin(["USD", "CAD"])]
current_na_na = current_na[current_na["currency"].isin(["USD", "CAD"])]
current_row_na  = current_row[current_row["currency"].isin(["USD", "CAD"])]

current_global_row = current_global[~current_global["currency"].isin(["USD", "CAD"])]
current_na_row = current_na[~current_na["currency"].isin(["USD", "CAD"])]
current_row_row  = current_row[~current_row["currency"].isin(["USD", "CAD"])]

current_global_na = add_quantiles(current_global_na)
current_na_na = add_quantiles(current_na_na)
current_row_na = add_quantiles(current_row_na)

current_global_row = add_quantiles(current_global_row)
current_na_row = add_quantiles(current_na_row)
current_row_row = add_quantiles(current_row_row)

current_global = add_quantiles(current_global)
current_na = add_quantiles(current_na)
current_row = add_quantiles(current_row)

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,0])

current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,1])

current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[0,2])

max_y = max(max(current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[0,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)


axs[0, 0].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[0, 1].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[0, 2].axhline(current_global[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[0, 0].set_ylabel('Global test', fontsize=16)

current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,0])
current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,1])
current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[1,2])

max_y = max(max(current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row_na.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[1,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)

axs[1, 0].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[1, 1].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[1, 2].axhline(current_global_na[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[1, 0].set_ylabel('NA test', fontsize=16)


current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,0])
current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,1])
current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean().plot(kind="bar", grid=True, ax=axs[2,2])

max_y = max(max(current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           max(current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))

min_y = min(min(current_global_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_na_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()),
           min(current_row_row.groupby("conviction_quantile")[target_variable + "_fwd"].mean()))
for ax in axs[2,:]:
    ax.set_ylim(min_y - 0.001, max_y + 0.001)

axs[2, 0].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[2, 1].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")
axs[2, 2].axhline(current_global_row[target_variable + "_fwd"].mean(), label="avg trr_5_fwd")

axs[2, 0].set_ylabel('ROW test', fontsize=16)

axs[0, 0].set_title("Global train", fontsize=16)
axs[0, 1].set_title("NA train", fontsize=16)
axs[0, 2].set_title("ROW train", fontsize=16)

plt.tight_layout()
fig.subplots_adjust(wspace=0.25)

for ax in axs.flat:
    ax.set_xlabel("")
    ax.tick_params(axis='both', which='major', labelsize=14)
    vals = ax.get_yticks()
    print(vals)
    ax.set_yticklabels(['+{:,.2%}'.format(abs(x)) if x > 0 else '{:,.2%}'.format(x) if x == 0 else '-{:,.2%}'.format(abs(x)) for x in vals])